# Resistance table Midday Spring

The objective of this script is to automatically generate the radiative fluxes table of this paper, and export it to LaTeX, with the proper t-tests.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

### Input and output paths

In [ ]:
project_path = './'
data_path   = project_path + '../../data/'

dataset_fn = data_path + 'dataset.csv'

#output_path  = project_path + 'output/'

In [ ]:
def load_data(fn, index_col=False, silent=False):
    if (not silent):
        print('-', fn.split('/')[-1])
    temp = pd.read_csv(fn, index_col=index_col)
    temp['DateTime'] = pd.to_datetime(temp['DateTime'], format='%Y-%m-%d %H:%M:%S', utc=True)
    return(temp)

def averaging(temp):
    
    # define mid-day
    temp = temp.loc[(temp['DateTime'].dt.hour >= 10) & (temp['DateTime'].dt.hour < 15)].copy()

    # Make mean and std dev
    df_means = temp.groupby(['Season','Ecosystem']).mean().reset_index()
    df_sds   = temp.groupby(['Season','Ecosystem']).std().reset_index()
    # rename columns
    df_means.rename(columns={'H': 'H_mean'}, inplace=True)
    df_means.rename(columns={'LE': 'LE_mean'}, inplace=True)
    df_means.rename(columns={'Rn': 'Rn_mean'}, inplace=True)
    df_means.rename(columns={'Ta': 'Ta_mean'}, inplace=True)
    df_means.rename(columns={'Ts': 'Ts_mean'}, inplace=True)
    df_means.rename(columns={'D_T': 'D_T_mean'}, inplace=True)
    df_means.rename(columns={'Pa': 'Pa_mean'}, inplace=True)
    df_means.rename(columns={'H2O': 'H2O_mean'}, inplace=True)
    df_means.rename(columns={'Lout': 'Lout_mean'}, inplace=True)
    df_means.rename(columns={'Lin': 'Lin_mean'}, inplace=True)
    df_means.rename(columns={'rho': 'rho_mean'}, inplace=True)
    df_means.rename(columns={'cp': 'cp_mean'}, inplace=True)
    df_means.rename(columns={'rH': 'rH_mean'}, inplace=True)
    df_means.rename(columns={'SWin': 'SWin_mean'}, inplace=True)
    df_means.rename(columns={'SWout': 'SWout_mean'}, inplace=True)

    df_sds.rename(columns={'H': 'H_sd'}, inplace=True)
    df_sds.rename(columns={'LE': 'LE_sd'}, inplace=True)
    df_sds.rename(columns={'Rn': 'Rn_sd'}, inplace=True)
    df_sds.rename(columns={'Ta': 'Ta_sd'}, inplace=True)
    df_sds.rename(columns={'Ts': 'Ts_sd'}, inplace=True)
    df_sds.rename(columns={'D_T': 'D_T_sd'}, inplace=True)
    df_sds.rename(columns={'Pa': 'Pa_sd'}, inplace=True)
    df_sds.rename(columns={'H2O': 'H2O_sd'}, inplace=True)
    df_sds.rename(columns={'Lout': 'Lout_sd'}, inplace=True)
    df_sds.rename(columns={'Lin': 'Lin_sd'}, inplace=True)
    df_sds.rename(columns={'rho': 'rho_sd'}, inplace=True)
    df_sds.rename(columns={'cp': 'cp_sd'}, inplace=True)
    df_sds.rename(columns={'rH': 'rH_sd'}, inplace=True)
    df_sds.rename(columns={'SWin': 'SWin_sd'}, inplace=True)
    df_sds.rename(columns={'SWout': 'SWout_sd'}, inplace=True)
    
    merged = df_means.merge(df_sds, on=['Season','Ecosystem'])
    
    # Keep only relevant columns
    merged = merged[['Season','Ecosystem','H_mean','LE_mean','Rn_mean','Ta_mean','Ts_mean','D_T_mean','Pa_mean','H2O_mean','Lout_mean','Lin_mean','rho_mean','cp_mean',\
                     'H_sd','LE_sd','Rn_sd','Ta_sd','Ts_sd','D_T_sd','Pa_sd','H2O_sd','Lout_sd','Lin_sd','rho_sd','cp_sd','rH_mean','rH_sd','SWin_mean','SWin_sd','SWout_mean','SWout_sd']]
    
    return(merged)


In [ ]:
# Load data
print('Load data:')
all_df = load_data(dataset_fn)

print('Done...')

In [ ]:
temp = all_df.copy()
# When both H and D_T are too small, ignore
temp.loc[(np.abs(temp['H']) < 5) | (np.abs(temp['D_T']) < 2), 'rH'] = np.nan
# When H and D_T are too close to each other as values, ignore
temp.loc[(np.abs(temp['H']) <= (np.abs(temp['D_T']) + 2)) &
                (np.abs(temp['H']) >= (np.abs(temp['D_T']) - 2)), 'rH'] = np.nan

# At night, we sometimes get rH < 0. Remove
temp.loc[(temp['rH'] < 0) & (temp['SWin'] < 5), 'rH'] = np.nan
# In fact, negative values are not supposed to be valid at all
temp.loc[(temp['rH'] < 0), 'rH'] = np.nan

mean_df = averaging(temp)

# Create a text of summarised values (mean + stddev)
mean_df['Rn'] = mean_df['Rn_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['Rn_sd'].astype(float).round(0).astype(int).astype(str) + ')'
mean_df['H'] = mean_df['H_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['H_sd'].astype(float).round(0).astype(int).astype(str) + ')'
mean_df['LE'] = mean_df['LE_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['LE_sd'].astype(float).round(0).astype(int).astype(str) + ')'
mean_df['Ta'] = mean_df['Ta_mean'].astype(float).round(1).astype(str) + ' (' + mean_df['Ta_sd'].round(1).astype(str) + ')'
mean_df['Ts'] = mean_df['Ts_mean'].astype(float).round(1).astype(str) + ' (' + mean_df['Ts_sd'].round(1).astype(str) + ')'
mean_df['D_T'] = mean_df['D_T_mean'].astype(float).round(1).astype(str) + ' (' + mean_df['D_T_sd'].round(1).astype(str) + ')'
mean_df['rH'] = mean_df['rH_mean'].astype(float).round(0).astype(str) + ' (' + mean_df['rH_sd'].astype(float).round(0).astype(str) + ')'
mean_df['Lout'] = mean_df['Lout_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['Lout_sd'].astype(float).round(0).astype(int).astype(str) + ')'
mean_df['Lin'] = mean_df['Lin_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['Lin_sd'].astype(float).round(0).astype(int).astype(str) + ')'
#mean_df['SWout'] = mean_df['SWout_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['SWout_sd'].astype(float).round(0).astype(int).astype(str) + ')'
#mean_df['SWin'] = mean_df['SWin_mean'].astype(float).round(0).astype(int).astype(str) + ' (' + mean_df['SWin_sd'].astype(float).round(0).astype(int).astype(str) + ')'

# Remove the original values
mean_df.drop(['H_mean','H_sd','LE_mean','LE_sd','Rn_mean','Rn_sd','Ta_mean','Ta_sd','Ts_mean','Ts_sd','D_T_mean','D_T_sd','Pa_mean','rho_mean','cp_mean',\
              'Pa_sd','H2O_mean','H2O_sd','Lout_mean','Lout_sd','Lin_mean','Lin_sd','rho_sd','cp_sd','rH_mean','rH_sd','SWin_mean','SWin_sd','SWout_mean','SWout_sd'], axis=1, inplace=True)
mean_df = mean_df.loc[mean_df['Season'] == 'Spring']

# Convert to long format
out_df = mean_df.pivot(index='Season', columns='Ecosystem').stack(level=[0])
out_df.reset_index(inplace=True)
out_df.drop(['Season'], axis=1, inplace=True)
out_df.rename(columns={'level_1': 'Parameter'}, inplace=True)
display(out_df)

#out_df.to_latex(output_path + 'Ketura_Science_resistance_spring.tex', index=False)